In [28]:
import datasets
from torch.utils.data import DataLoader
import torch
import os
import json
from torchvision import transforms
from tqdm import tqdm
import glob
from PIL import Image
import matplotlib.pyplot as plt

from sache import SpecifiedHookedViT

Matplotlib is building the font cache; this may take a moment.


In [17]:
ds = datasets.load_dataset("lmms-lab/GQA", 'submission_all_images')['submission']

In [38]:


# Define the transformation for PIL images (e.g., resize, convert to tensor)
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL image to a tensor
])

def collate_fn(batch):
    images = []
    ids = []
    for item in batch:
        ids.append(item['id'])
        img = item['image']
        img = img.resize((224, 224))
        images.append(transform(img))
    images = torch.stack(images)  # Stack images into a single tensor
    
    return ids, images

# Update DataLoader to use the custom collation function
batch_size = 1024
dataloader = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=collate_fn)


In [39]:
def generate_latents(
        sae_path,
        dataloader,
        n_activations=250_000,
        save_every=10,
        batch_size=2048,
        transformer_name='laion/CLIP-ViT-L-14-laion2B-s32B-b82K',
        hook_name="resid",
        layer=22,
        device='cuda',
    ):

    data_directory = 'images'
    
    location = (layer, hook_name)
    transformer = SpecifiedHookedViT([location], transformer_name, device=device)
    sae = torch.load(sae_path)
    
    n_steps = sae_path.split('/')[-1].split('.')[0]
    latent_dir = "/".join(sae_path.split('/')[:-1]) + f'/latents-{n_steps}'

    if not os.path.exists(latent_dir):
        os.makedirs(latent_dir)

    latents = None
    file_paths = []
    with torch.no_grad():
        for i, (paths, batch) in tqdm(enumerate(dataloader), total=n_activations // batch_size):
            activations = transformer.cls_activations(batch)[location]

            latent = sae.forward_descriptive(activations)['latent'].detach().cpu()

            if latents is None:
                latents = latent
            else:
                latents = torch.cat((latents, latent), dim=0)

            file_paths.extend(paths)


            if i > 0 and i % save_every == 0:
                with open(f'{latent_dir}/file_paths_{i}.json', 'w') as f:
                    json.dump(file_paths, f)
                torch.save(latents, f'{latent_dir}/latents_{i}.pt')
                file_paths = []
                latents = None

            if i * batch_size > n_activations:
                if len(file_paths) > 0:
                    with open(f'{latent_dir}/file_paths_{i}.json', 'w') as f:
                        json.dump(file_paths, f)
                    torch.save(latents, f'{latent_dir}/latents_{i}.pt')
                break

    return latent_dir

In [40]:
generate_latents(
    sae_path='../vitact/cruft/3026432.pt',
    dataloader=dataloader,
    batch_size=1024
)

/tmp/ipykernel_42440/308672012.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sae = torch.load(sae_path)
huggingface/tokenizers: The current process just got forked, a

In [36]:
def get_top9(latent_dir):
    file_path_files = sorted(glob.glob(os.path.join(latent_dir, 'file_paths_*.json')))
    latent_files = sorted(glob.glob(os.path.join(latent_dir, 'latents_*.pt')))

    image_dir = os.path.join(latent_dir, 'images')

    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
    print('saving images to:', image_dir)

    num_features = 650
    num_top = 9  # We need top 9 images per feature

    # Initialize topk values and indices for all features
    topk_values = torch.full((num_features, num_top), float('-inf'))
    topk_indices = torch.full((num_features, num_top), -1, dtype=torch.long)
    topk_file_paths = [[''] * num_top for _ in range(num_features)]
    indices_gt_zero = [[] for _ in range(num_features)]

    cumulative_index = 0  # Keeps track of the global index across batches
    cumulative_file_paths = []

    for batch_num, (fp_file, latent_file) in enumerate(zip(file_path_files, latent_files)):
        print(f'Processing batch {batch_num+1}/{len(latent_files)}')
        with open(fp_file, 'r') as f:
            file_paths = json.load(f)
        latents = torch.load(latent_file)[:, :650]  # shape: [batch_size, num_features]

        batch_size = latents.size(0)
        current_indices = torch.arange(cumulative_index, cumulative_index + batch_size, dtype=torch.long)
        cumulative_index += batch_size

        cumulative_file_paths.extend(file_paths)

        # Transpose latents to shape [num_features, batch_size]
        current_values = latents.t()  # shape: [num_features, batch_size]

        # Expand current_indices to shape [num_features, batch_size]
        current_indices_expanded = current_indices.unsqueeze(0).expand(num_features, -1)  # shape: [num_features, batch_size]

        # Concatenate current batch with previous topk
        total_values = torch.cat((topk_values, current_values), dim=1)  # shape: [num_features, num_top + batch_size]
        total_indices = torch.cat((topk_indices, current_indices_expanded), dim=1)  # shape: [num_features, num_top + batch_size]

        # Perform topk across the concatenated values
        topk = torch.topk(total_values, k=num_top, dim=1)
        topk_values = topk.values  # shape: [num_features, num_top]
        indices_in_total = topk.indices  # shape: [num_features, num_top]

        # Gather the corresponding global indices
        topk_global_indices = torch.gather(total_indices, 1, indices_in_total)  # shape: [num_features, num_top]

        # Update topk_indices
        topk_indices = topk_global_indices

        # Update topk_file_paths
        for feature_idx in range(num_features):
            indices_in_total_feature = indices_in_total[feature_idx]
            total_file_paths = topk_file_paths[feature_idx] + file_paths
            topk_file_paths[feature_idx] = [total_file_paths[i] for i in indices_in_total_feature.tolist()]

        # Update indices_gt_zero
        mask = current_values > 0  # shape: [num_features, batch_size]
        for feature_idx in range(num_features):
            indices = current_indices[mask[feature_idx]]
            indices_gt_zero[feature_idx].extend(indices.tolist())

    # After processing all batches, save the results
    for feature_idx in tqdm(range(num_features)):

        if topk_values[feature_idx].min() <= 0:
            continue

        result = {
            'indices': topk_indices[feature_idx].tolist(),
            'values': topk_values[feature_idx].tolist(),
            'indices_gt_zero': indices_gt_zero[feature_idx],
            'file_paths': topk_file_paths[feature_idx]
        }

        feature_dir = os.path.join(image_dir, f'feature_{feature_idx}')
        if not os.path.exists(feature_dir):
            os.makedirs(feature_dir)

        with open(os.path.join(feature_dir, f'{feature_idx}_top9.json'), 'w') as f:
            json.dump(result, f)

        # Load the images
        images = []
        for i, path in enumerate(topk_file_paths[feature_idx]):
            if i >= num_top:
                break
            img = Image.open(path).convert('RGB')
            images.append(img)

            # Save image
            img.save(os.path.join(feature_dir, f'{feature_idx}_top9_{i}.png'))

        # Plot the images in a 3x3 grid
        fig, axes = plt.subplots(3, 3, figsize=(9, 9))
        for ax, img in zip(axes.flatten(), images):
            ax.imshow(img)
            ax.axis('off')
        plt.tight_layout()
        plt.savefig(os.path.join(feature_dir, f'{feature_idx}_grid.png'))
        plt.close(fig)

        print(f'Saved images for feature {feature_idx}')


In [37]:
get_top9('cruft/latents-3026432/')

saving images to: cruft/latents-3026432/images


100%|██████████| 650/650 [00:00<00:00, 36034.02it/s]
